In [1]:
from main import init_system
from api.apiutils import Relation
api = init_system("/Users/ra-mit/development/discovery_proto/test/chembl21_drugbank_chebi_bindingdb.pickle")

Loading: */Users/ra-mit/development/discovery_proto/test/chembl21_drugbank_chebi_bindingdb.pickle*

### Help Menu

You can use the system through an **API** object. API objects are returnedby the *init_system* function, so you can get one by doing:

***your_api_object = init_system('path_to_stored_model')***

Once you have access to an API object there are a few concepts that are useful to use the API. **content** refers to actual values of a given field. For example, if you have a table with an attribute called __Name__ and values *Olu, Mike, Sam*, content refers to the actual values, e.g. Mike, Sam, Olu.

**schema** refers to the name of a given field. In the previous example, schema refers to the word__Name__ as that's how the field is called.

Finally, **entity** refers to the *semantic type* of the content. This is in experimental state. For the previous example it would return *'person'* as that's what those names refer to.

Certain functions require a *field* as input. In general a field is specified by the source name (e.g. table name) and the field name (e.g. attribute name). For example, if we are interested in finding content similar to the one of the attribute *year* in the table *Employee* we can provide the field in the following way:

field = ('Employee', 'year') # field = [<source_name>, <field_name>)


In [3]:
res = api.keyword_search("caffeine")
res.print_columns()

Hit(nid='1165321500', source_name='drug_interactions', field_name='description', score=0.7691053)
Hit(nid='147483850', source_name='formulations', field_name='ingredient', score=0.71199304)
Hit(nid='147483850', source_name='formulations', field_name='ingredient', score=0.5938432)
Hit(nid='147483850', source_name='formulations', field_name='ingredient', score=0.56489724)
Hit(nid='-397740983', source_name='molecule_synonyms', field_name='synonyms', score=0.56039596)
Hit(nid='-397740983', source_name='molecule_synonyms', field_name='synonyms', score=0.528413)
Hit(nid='427418858', source_name='products', field_name='trade_name', score=0.50023043)
Hit(nid='427418858', source_name='products', field_name='trade_name', score=0.4040715)
Hit(nid='147483850', source_name='formulations', field_name='ingredient', score=0.4040715)
Hit(nid='1661243416', source_name='molecule_dictionary', field_name='pref_name', score=0.40260035)


In [11]:
table = "molecule_dictionary"
drs_table = api.drs_from_table(table)
similar_tables = api.similar_content_to(drs_table)
similar_tables.print_tables()

ontology
target_components
target_dictionary
vertice
ligand_eff
metabolism
products
usan_stems
molecule_dictionary
drug_mechanism
compound_properties
assays
product_patents


In [12]:
table = "molecule_dictionary"
drs_table = api.drs_from_table(table)
similar_tables = api.similar_schema_name_to(drs_table)
similar_tables.print_tables()

biotherapeutics
formulations
compound_records
drug_mechanism
protein_classification
molecule_frac_classification
target_dictionary
biotherapeutic_components
compound_properties
assays
molecule_hrac_classification
chembl_id_lookup
molecule_atc_classification
compound_structural_alerts
molecule_synonyms
go_classification
cell_dictionary
products
molecule_irac_classification
drug_indication
docs
molecule_hierarchy


In [16]:
field = ("drug_interactions", "description")
drs_field = api.drs_from_raw_field(field)
similar = api.similar_schema_name_to(drs_field)
similar.print_columns()

Hit(nid=1424199745, source_name='data_validity_lookup', field_name='description', score=0.0)
Hit(nid=3941335841, source_name='drugs_vw', field_name='description', score=0.0)
Hit(nid=1106713799, source_name='biotherapeutics', field_name='description', score=0.0)
Hit(nid=3890889352, source_name='drugs', field_name='description', score=0.0)
Hit(nid=845373320, source_name='parameter_type', field_name='description', score=0.0)
Hit(nid=1891512332, source_name='action_type', field_name='description', score=0.0)
Hit(nid=408355149, source_name='confidence_score_lookup', field_name='description', score=0.0)
Hit(nid=20714259, source_name='bio_component_sequences', field_name='description', score=0.0)
Hit(nid=1490727572, source_name='curation_lookup', field_name='description', score=0.0)
Hit(nid=2770821171, source_name='assays', field_name='description', score=0.0)
Hit(nid=2642382261, source_name='component_sequences', field_name='description', score=0.0)
